In [68]:
import matplotlib.pyplot as plt
import polars as pl
import pandas as pd
import numpy as np
import json
import os
import sys

sys.path.append("..")

import seaborn as sns

from settings import (
    random_state,
    PROJECT_PATH,
    REGRESSION_TARGET,
    CLASSIFICATION_TARGET,
)
from catboost import CatBoostClassifier, Pool
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import recall_score, precision_score, f1_score
import mlflow
from mlflow import MlflowClient

Ce corrigé se base sur la version suivante de Mlflow

In [2]:
mlflow.__version__

'2.15.1'

In [3]:
def perform_cross_validation(
    X: pl.DataFrame,
    y: pl.Series,
    model,
    cross_val_type,
    scoring_metrics: tuple,
    groups=None,
):
    scores = cross_validate(
        model,
        X.to_numpy(),
        y.to_numpy(),
        cv=cross_val_type,
        return_train_score=True,
        return_estimator=True,
        scoring=scoring_metrics,
        groups=groups,
    )

    scores_dict = {}
    for metric in scoring_metrics:
        scores_dict["average_train_" + metric] = np.mean(scores["train_" + metric])
        scores_dict["train_" + metric + "_std"] = np.std(scores["train_" + metric])
        scores_dict["average_test_" + metric] = np.mean(scores["test_" + metric])
        scores_dict["test_" + metric + "_std"] = np.std(scores["test_" + metric])

    model.fit(X.to_numpy(), y.to_numpy())

    return scores, scores_dict, model

def get_features_most_importance(importances, feature_names, threshold=0.8):
    sorted_indices = np.argsort(importances)
    sorted_importances = importances[sorted_indices][::-1]
    sorted_feature_names = [feature_names[i] for i in sorted_indices][::-1]

    cumulated_importance = 0
    important_features = []

    for importance, feature_name in zip(sorted_importances, sorted_feature_names):
        cumulated_importance += importance
        important_features.append(feature_name)

        if cumulated_importance >= threshold:
            break

    return important_features

In [4]:
transactions = pl.read_parquet(
    os.path.join(PROJECT_PATH, "transactions_post_feature_engineering.parquet")
)


with open("../features_used.json", "r") as f:
    feature_names = json.load(f)

with open("../categorical_features_used.json", "r") as f:
    categorical_features = json.load(f)

numerical_features = [col for col in feature_names if col not in categorical_features]

In [90]:
transactions_v1 = transactions.filter(pl.col("annee_transaction") < 2020)

transactions_v2 = transactions.filter(
    pl.col("annee_transaction").is_between(2020, 2021)
)
features_1 = [
    "type_batiment_Appartement",
    "surface_habitable",
    "prix_m2_moyen_mois_precedent",
    "nb_transactions_mois_precedent",
    "taux_interet",
    "variation_taux_interet",
    "acceleration_taux_interet",
]

features_2 = features_1 + ["longitude", "latitude", "vefa"]

<b> Attention : </b> Assurez-vous de lancer ce code dans le même dossier où les modèles pré-covid ont été créés ! En effet, quand vous lancez votre MLflow UI (ainsi qu'un objet MlflowClient) sans arguments supplémentaires, le package va utiliser comme modèle registry ceux du dossier actuel. Plus précisement, Mlflow va créer un dossier mlruns, où seront stockés tous vos modèles. 

Il suffit alors de : 
* lancer dans votre Terminal la commande mlflow ui en précisant l'adresse du dossier mlruns comme ceci : mlflow ui --backend-store-uri adresse_de_votre_choix
* Réaliser la même opération côté Python avec la commande mlflow.set_tracking_uri("adresse_de_votre_choix")

Nous déclarons de nouvelles valeurs de tags pour rester cohérent avec le contexte : Nouveau feature engineering post-covid

In [6]:
experiment_tags = {
    "region": "Nouvelle-Aquitaine",
    "revision_de_donnees": "v2",
    "date_de_construction": "Fin 2021",
}

Nous précisons à MLflow que nous allons travailler avec l'experiment Nouvelle-Aquitaine comme vu dans le cours.

In [53]:

client = MlflowClient(tracking_uri="http://127.0.0.1:5000")
nouvelle_acquitaine_experiment = mlflow.set_experiment("Nouvelle-Aquitaine")



Nous pouvons récupérer l'experiment id en printant l'objet Experiment

In [19]:
nouvelle_acquitaine_experiment_id = '247887458207936819'

In [10]:
transactions_post_covid_nouvelle_acquitaine = transactions_v2.filter(
    pl.col("nom_region_Nouvelle-Aquitaine") == 1
)

Pour pouvoir comparer des situations comparables, il faut d'abord charger les modèles entrainés sur la période pré-covid, réaliser une inférence sur la nouvelle donnée et observer les résultats. En effet, cela aurait peu de sens de comparer un nouveau modèle entrainé sur une nouvelle donnée avec un ancien modèle entrainé sur une ancienne donnée. 

Pour cela, nous utilisons la fonctionnalité search_runs. Nous allons utiliser l'experiment id pour trouver l'URI (adresse de stockage) du modèle avec les meilleures performances sur la période pré-covid. 

In [59]:
# Il s'agit d'un DataFrame 
all_experiments = mlflow.search_runs(search_all_experiments=True)

In [60]:
all_experiments.columns

Index(['run_id', 'experiment_id', 'status', 'artifact_uri', 'start_time',
       'end_time', 'metrics.train_precision_std',
       'metrics.average_train_recall', 'metrics.average_test_precision',
       'metrics.average_test_recall', 'metrics.train_recall_std',
       'metrics.average_train_precision', 'metrics.test_precision_std',
       'metrics.test_f1_std', 'metrics.average_train_f1',
       'metrics.average_test_f1', 'metrics.train_f1_std',
       'metrics.test_recall_std', 'params.features', 'params.random_state',
       'tags.mlflow.user', 'tags.mlflow.source.name',
       'tags.mlflow.log-model.history', 'tags.mlflow.runName',
       'tags.mlflow.source.type'],
      dtype='object')

Pour illustrer ici, nous nous contentons d'une approche simple où l'on choisit le modèle avec le meilleure f1_score moyen. Une approche plus robuste consisterait à comparer les scores en train et test pour vérifier la part d'overfit et regarder l'ecart-type des scores. 

In [24]:
highest_f1_score = all_experiments["metrics.average_test_f1"].max()

In [25]:
best_precovid_model = all_experiments.loc[
    (all_experiments["experiment_id"] == nouvelle_acquitaine_experiment_id) 
    & (all_experiments["metrics.average_test_f1"] == highest_f1_score)
    
]

Nous recupérons ensuite le Run ID du modèle, à partir duquel nous pouvons déduire son adresse. 

In [44]:
best_precovid_model_run_id = best_precovid_model["run_id"].values[0] # En cas de plusieurs modèles avec la même performance

Cette commande d'identifier le nom du fichier du modèle. C'est le même nom que celui qui a été stocké par la méthode log_model

In [62]:
client.list_artifacts(best_precovid_model_run_id)

[<FileInfo: file_size=None, is_dir=True, path='catboost_classifier'>]

In [64]:
model_local_path = mlflow.artifacts.download_artifacts(
  run_id=best_precovid_model_run_id, artifact_path="catboost_classifier"
)

In [66]:
best_precovid_model = mlflow.sklearn.load_model(model_local_path)

Nous définissons alors un Run qui va réaliser une inférence du modèle pre-covid sur la donnée post-covid.

In [72]:

with mlflow.start_run(run_name="pre_covid_catboost_Nouvelle-Aquitaine_post_covid_data") as run:

    X = transactions_post_covid_nouvelle_acquitaine.drop(
        [REGRESSION_TARGET, CLASSIFICATION_TARGET]
    ).to_pandas()
    y_classification = transactions_post_covid_nouvelle_acquitaine[CLASSIFICATION_TARGET].to_pandas()

    classification_scoring_metrics = ["recall", "precision", "f1"]

    predictions = best_precovid_model.predict(Pool(X[features_1],y_classification))

    scores_dict = {}
    scores_dict["recall"] = recall_score(y_classification, predictions)
    scores_dict["precision"] = precision_score(y_classification, predictions)
    scores_dict["f1"] = f1_score(y_classification, predictions)

    mlflow.log_param("random_state", random_state)
    mlflow.log_param("features", features_1)

    for metric, value in scores_dict.items():
        mlflow.log_metric(metric, value)

    mlflow.sklearn.log_model(best_precovid_model, "best_pre_covid_model_post_covid_data")

    dataset_abstraction = mlflow.data.from_pandas(
        transactions_post_covid_nouvelle_acquitaine.to_pandas()
    )
    mlflow.log_input(dataset_abstraction)

2024/10/28 17:53:46 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
c:\Users\Zakaria\Desktop\Data Science\Openclassrooms\supervisedlearning\.venv\Lib\site-packages\mlflow\types\utils.py:406: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [85]:
run_metrics = mlflow.search_runs(
    filter_string="""
    tags.mlflow.runName = 'pre_covid_catboost_Nouvelle-Aquitaine_post_covid_data' 
    AND status = 'FINISHED'
    """
)
run_metrics = run_metrics[[col for col in run_metrics.columns if col.startswith("metrics.")]]

In [86]:
run_metrics

,metrics.f1,metrics.precision,metrics.recall
0,0.515822,0.650209,0.427471


Nous réalisons ensuite notre Run de la même manière que dans le cours, en utilisant le nouveau jeu de features et de données.

In [89]:
features_2

In [91]:

with mlflow.start_run(run_name="catboost_Nouvelle-Aquitaine_post_covid") as run:

    X = transactions_post_covid_nouvelle_acquitaine.drop(
        [REGRESSION_TARGET, CLASSIFICATION_TARGET]
    ).to_pandas()
    y_classification = transactions_post_covid_nouvelle_acquitaine[CLASSIFICATION_TARGET].to_pandas()

    catboost_model = CatBoostClassifier(random_state=random_state, verbose=False)
    classification_scoring_metrics = ["recall", "precision", "f1"]

    scores, scores_dict, catboost_model = perform_cross_validation(
        X=X[features_2],
        y=y_classification,
        model=catboost_model,
        cross_val_type=StratifiedKFold(),
        scoring_metrics=classification_scoring_metrics,
    )

    mlflow.log_param("random_state", random_state)
    mlflow.log_param("features", features_2)

    for metric, value in scores_dict.items():
        mlflow.log_metric(metric, value)

    mlflow.sklearn.log_model(catboost_model, "catboost_classifier_post_covid")

    dataset_abstraction = mlflow.data.from_pandas(
        transactions_post_covid_nouvelle_acquitaine.to_pandas()
    )
    mlflow.log_input(dataset_abstraction)

feature_importances = catboost_model.get_feature_importance(Pool(X[features_2]))
most_important_features = get_features_most_importance(
    feature_importances, features_2
)


2024/10/28 18:11:08 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
c:\Users\Zakaria\Desktop\Data Science\Openclassrooms\supervisedlearning\.venv\Lib\site-packages\mlflow\types\utils.py:406: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Au lieu d'utiliser le MLflow UI, nous allons programmatiquement chercher le meilleur modèle parmi ceux post covid et ceux pré-covid. 

In [93]:
run_metrics_new_model = mlflow.search_runs(
    filter_string="""
    tags.mlflow.runName = 'catboost_Nouvelle-Aquitaine_post_covid'
    AND status = 'FINISHED'
    """
)
run_metrics_new_model = run_metrics_new_model[[col for col in run_metrics_new_model.columns if col.startswith("metrics.")]]

In [94]:
run_metrics_new_model

,metrics.train_precision_std,metrics.average_train_recall,metrics.average_test_precision,metrics.average_test_recall,metrics.train_recall_std,metrics.average_train_precision,metrics.test_precision_std,metrics.test_f1_std,metrics.average_train_f1,metrics.average_test_f1,metrics.train_f1_std,metrics.test_recall_std
0,0.006768,0.552081,0.436451,0.465252,0.01664,0.819325,0.101887,0.10322,0.659542,0.442434,0.012896,0.138184
